### ranges

[std ranges (v3) (since C++20)](https://en.cppreference.com/w/cpp/ranges)

[boost ranges (v2)](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/index.html)

**Замечание:** на данный момент ranges-v3 ещё не устаканились и в стандарте их нет, но попробовать уже можно:
* [github-репозиторий reicneibler range v3](https://github.com/ericniebler/range-v3) - header-only библиотека рэнжей, многое из неё попало в стандарт
* [документация по range v3](https://ericniebler.github.io/range-v3/)

<br />

#####  Откуда пошли и зачем нужны ranges

В STL набор элементов всегда предсатвлена парой итераторов `[begin, end)`.

Поэтому код сортировки или проверки нахождения элемента в массиве выглядят так:

```c++
std::vector<std::string> students_in_the_class = ...;
std::sort(students_in_the_class.begin(), students_in_the_class.end());

if (std::find(students_in_the_class.begin(),
              students_in_the_class.end(),
              "Those boy Einstein") != students_in_the_class.end())
{
    std::cout << "Oh my God!" << std::endl;
}
```

А хотелось бы чего-нибудь такого:

```c++
std::vector<std::string> students_in_the_class = ...;
std::sort(students_in_the_class);  // won't compile

if (std::contains(students_in_the_class, "Those boy Einstein"))  // won't compile
{
    std::cout << "Oh my God!" << std::endl;
}
```

<br />

Первая же идея, которая часто приходит в голову: давайте сделаем шаблонный тип - последовательность, хранящий в себе произвольную пару `[begin, end)`, и научим алгоритмы работать с ним. Вот мы и придумали простой вариант `Range`.

Но этим все желания не ограничиваются. Что хочется ещё:

1. действия над объектами (фильтрация, преобразование ...):

    ```c++
    auto numbers_rng = /* range of integers: 1, 2, 3, 4, 5 */;
    auto even_sqr_numbers = numbers_rns.filter(is_even).transform(sqr); /* range: 4, 16 */
    ```

2. чтобы программист решал, хочет он ленивое вычисление `even_sqr_numbers` или вычисление всего сразу

    в таком варианте хочется не вычислять 500 квадратов, а найти первый и остановиться

    ```c++
    {
        auto numbers_rng = /* range of integers: 1, 2, 3, 4, 5, 6, .... 1000 */;
        auto even_sqr_numbers = numbers_rns.filter(is_even).transform(sqr); /* range: 4, 16, 36, ... */
        std::cout << *even_sqr_numbers.begin();
    }
    ```
    
3. читабельность / производительность / расширяемость / быструю компиляцию / принцессу / полцарства / быть владычицей морскою

<br />

##### Мотивационные примеры

Продемонстрируем возможности std::ranges (v3) по сравнению с обычной библиотекой алгоритмов.

Определим пару вспомогательных вещей:

```c++
namespace rs = ranges::v3;
namespace rv = ranges::v3::view;
namespace ra = ranges::v3::action;

std::vector<int> v = {1, 1, 2, 3, 5, 8, 13, 21, 34};

bool is_even(int x) {
    return x % 2 == 0;
}

void print_item(int x) {
  std::cout << x << std::endl;
}
```

<br />

**Задача**: распечатать все элементы вектора в обратном порядке:

```c++
// std
std::for_each(v.rbegin(), v.rend(), print_item);

// ranges
for (int x : v | rv::reverse)
    print_item(x);
```


<br />

**Задача**: распечатать все чётные элементы вектора в обратном порядке:

```c++
// std
std::for_each(v.rbegin(), v.rend(),
              [&](int x) {
                  if (is_even(x))
                      print_item(x);
              });

// ranges
for (int x : v | rv::reverse | rv::filter(is_even))
   print_elem(x);
```

<br />

**Задача**: положить в вектор квадраты чётных чисел от 1 до 100

```c++
// std
std::vector<int> v;
for (int n = 1; n < 101; ++n)
    if (is_even(n))
        v.push_back(sqr(n));

// ranges
auto v = rs::iota_view(1, 101) | rv::filter(is_even) | rv::transform(sqr) | rv::to<std::vector>;
```

<br />

**Задача**: положить в вектор три наибольших числа из промежутка `[100, 200]`, делящиеся на 7, в порядке убывания

```c++
// std
std::vector<int> v;
for (int n = 200, count = 0; n >= 100 && count < 3; --n)
{
    if (n % 7 == 0)
    {
        v.push_back(n);
        ++count;
    }
}

// ranges
auto v = rs::iota_view(100, 201)
       | rv::reverse
       | rv::filter([](int x){ return x % 7 == 0; })
       | rv::take(3)
       | rs::to<std::vector>;
```

<br />

**Задача**: отсортировать вектор по убыванию и оставить уникальные элементы

```c++
// std
std::sort(v.rbegin(), v.rend());
v.erase(std::unique(v.begin(), v.end()), v.end());
std::reverse(v.begin(), v.end());

// ranges
v = std::move(v) | ra::sort |  ra::unique | ra::reverse;
```

<br />

**Задача**: собрать все ключи отображения в вектор

```c++
std::map<int, std::string> m;

// std
std::vector<int> v;
for (const auto& [k, _]: m)
    v.push_back(k);

// ranges
auto v = m | rv::keys | rs::to<std::vector>;
```

<br />

##### views are lazy, actions are eager

Кроме `range` в range-v3 ещё есть `view` и `action`.

Их основное отличие:
* `view` вычисляют результат лениво
* `action` вычисляют результат сразу

С `action` всё более-менее понятно:

```c++
std::vector<std::string> v1;
auto v2 = v1 | ra::sort | ra::unique;

// v2 - готовый std::vector<std::string>
```

<br />

Разберёмся с lazy-свойством у `view`:

```c++
std::vector<int> v = {1, 2, 3, 4, 5};
auto even_numbers = v | rv::reverse | rv::filter(is_even);
// к этой строке функция |is_even| не была вызвана ни разу

std::cout << *rg::begin(even_numbers);
// Вопрос: сколько раз is_even должен позваться здесь?
```

**Вопрос:**

* как можно было бы организовать `rv::reverse` внутри, чтобы обеспечить ленивое вычисление
* как можно было бы организовать `rv::filter` внутри, чтобы обеспечить ленивое вычисление

<details>
<summary>возможный ответ</summary>
    
* `reverse`-объект (адаптер) хранит ссылку на ренж, который ему пришёл
* `filter`-объект (адаптер) хранит ссылку на ренж, который ему пришёл
* у `reverse` и у `filter` будут спрашивать итераторы, когда по ним будут итерироваться
* `reverse` должен вернуть reverse iterator
* `filter` должен вернуть особый итератор, который в своём `operator++` будет пропускать элементы исходного рэнжа, пока не выполнится условие
    
</details>

**Вопрос:** что не так с этим кодом?

```c++
auto generate_even_numbers_view(int max_num)
{
    std::vector<int> v;
    for (int i = 0; i <= max_num; ++i)
        v.push_back(i);
    
    return v | rv::filter(is_even);
}
```

**Вопрос:** что-нибудь напоминает?

<br />

Пока ещё стандарт не проработан до конца, поэтому на cppreference документация бедная.

Искать какие есть view и action для range можно [тут](https://ericniebler.github.io/range-v3/)

Также вы можете создавать собственные `view` / `action`, но это немного хардкорно сделано в ranges-v3, и поэтому не войдёт в рамки лекции.

<br />

##### ranges-v3 features: generalized callables && projections

Полный набор отличий Эрик Ниблер (автор ranges) описывает в своей обзорной статье:

http://ericniebler.com/2018/12/05/standard-ranges/

Отдельно хотелось бы остановиться на паре фичей: generalized callables && projections

<br />

Generalized callables - фича, позволяющая передавать в range-алгоритмы конструкции, которые нельзя передавать в обычные алгоритмы.

Например, указатель на метод класса нельзя передать в обычный алгоритм, а в range-алгоритмы можно:

```c++
class Widget {
public:
  void hide();
};

std::vector<Widget> widgets { /*...*/ };

// compiler error
// std::for_each(widgets.begin(), widgets.end(), &Widget::hide);

// ok for ranges
rg::for_each(widgets, &Widget::hide);
```

**Вопрос:** скомпилируется ли такой код?

```c++
class Widget {
public:
  void set_visible(bool is_visible);
};

std::vector<Widget> widgets { /*...*/ };
rg::for_each(widgets, &Widget::set_visible);
```

<details>
<summary>ещё вопрос</summary>
    
а как сделать чтобы скомпилировался?
    
</details>

<br />

Projections - возможность "проецировать" объекты в алгоритмах через callable.

Например: подставлять указатель на поле класса (проекция всего объекта на какое-то из его полей).

```c++
struct Person
{
  std::size_t age;
  std::string name;
};

std::vector<Person> people = { /*...*/ };

// отсортировать людей по имени
rs::sort(people, rg::less{}, &Person::name);

// проверить, что есть человек Добрыня
std::cout << "Is Dobrynia with us: " << rs::contains(people, "Dobrynia", &Person::name));
```

Например: преобразовать объект в другой объект через функцию / лямбду (проекция как функция в другое пространство)
        
```c++
struct Point
{
    float x;
    float y;
};

std::array<Point> points = { /*...*/ };

// найти точку с расстоянием до начала координат равным 5
auto it = rs::find(points, 5, [](Point p) { return sqrt(p.x*p.x + p.y*p.y); });
```    

<br />

Как реализуются projections в самой библиотеке рэнжей на примере `std::ranges::contains` (обратите внимание на параметр `proj`):
    
```c++
template<typename Rng, typename T, typename P = identity>
constexpr auto RANGES_FUNC(contains)(Rng && rng, const T & val, P proj = {})
    ->CPP_ret(bool)( //
        requires input_range<Rng> &&
        indirect_relation<equal_to, projected<iterator_t<Rng>, P>, const T *>)
{
    return (*this)(begin(rng), end(rng), val, std::move(proj));
}
```

<br />

**Замечание**: вдруг это не заметно с первого взгляда, простейший алгоритм `contains` содержит клубок шаблонной магии; как можете догадаться, при использовании библиотеки ranges скорость компиляции оставляет желать лучшего.

<br />

##### range generators

Билиотека предоставляет тривиальные генераторы ренжей. Ходят слухи по сети интернет, что они бывают полезны:
    
```c++
rv::iota(i)       // generates infinite range [i, i+1, i+2, i+3, ...)
rv::iota(i, j)    // generates [i, i+1, i+2, ..., j-1] (or [i,j))
rv::single(x)     // generates range of single element x
rv::empty<T>      // generates empty range
rv::counted(x, n) // generates range [x, x, ..., x] of size n
```

Если с конечными рэнжами всё более-менее понятно, то бесконечный ренж `rv::iota(i)` не сразу понятно как использовать.

Напомню, что числа он генерирует ленивым образом. Т.е. пока его не попросили, новое число он не посчиитает.

Логично, что пытаться сконструировать из него `std::vector` - так себе идея. Надо как-то обрывать его работу. Можно сделать это, например, через view-адаптер `take(n)`:

```c++
auto v = rv::iota(1)
       | rv::filter(is_even)
       | rv::transform(sqr)
       | rv::take(6)
       | rs::to<std::vector>();
```

<br />

##### Примеры того, когда НЕ надо использовать рэнжи

Сайт Эрика Ниблера содержит хорошие примеры с рэнжами.

На мой субъективный вгляд, часть из них - примеры того, когда рэнжи вредны, и вместо них лучше было бы написать стандартный алгоритм. Но у других программистов может быть другое представление о прекрасном.

**Пример**: сумма квадратов первых 10 чисел:

```c++
// ranges
const int sum = rs::accumulate(rv::ints(1)
                               | rv::transform([](int i){ return i*i; })
                               | rv::take(10), 0);

// std
int sum = 0;
for (int i = 1; i < 11; ++i)
    sum += i * i;
```


**Пример**: генерация `vector<int>`, где элемент `i` повторён `i` раз, `v == {1,2,2,3,3,3,4,4,4,4,5,5,5,5,5,...}`:

```c++
// ranges
const auto v = rv::for_each(rv::ints(1, 10),
                            [](int i) {
                                return yield_from(rv::repeat_n(i, i));
                            })
               | to<std::vector>();

// std
std::vector<int> v;
for (int i = 1; i < 10; ++i)
    for (int x = 0; x < i; ++x)
        v.push_back(i);
```

**Пример**: Пифагоровы тройки чисел тут http://ericniebler.com/2018/12/05/standard-ranges/

**Замечание**: в каждом конкретном случае руководствуйтесь своим собственным представлением о прекрасном (другого у вас, скорее всего, нет). В первую очередь - простота и читаемость.

<br />

##### TPOIASI problem

https://www.fluentcpp.com/2019/02/12/the-terrible-problem-of-incrementing-a-smart-iterator/

https://www.fluentcpp.com/2019/04/16/an-alternative-design-to-iterators-and-ranges-using-stdoptional/

https://www.fluentcpp.com/2019/02/15/how-smart-output-iterators-fare-with-the-terrible-problem-of-incrementing-a-smart-iterator/

TPOIASI - шуточное называние проблемы лишних вычислений при работе с ranges.

TPOIASI = Terrible Problem Of Incrementing A Smart Iterator

Иллюстрация проблемы:
    
```c++
int transform_fn(int x)
{
    std::cout << "transform is called for " << x << std::end;
    return x * 2;
}

bool filter_fn(int x)
{
    return x % 4 == 0;
}

std::vector<int> numbers = {1, 2, 3, 4, 5};
std::vector<int> results;
for (int x : v | rv::transform(transform_fn)
               | rv::filter(filter_fn))
     results.push_back(x);
```

В результате отработки кода будет выведено:
    
```sh
transform is called for 1
transform is called for 2  # !!!
transform is called for 2  # !!!
transform is called for 3
transform is called for 4  # !!!
transform is called for 4  # !!!
transform is called for 5
```

Проблема в том, что при проходе по рэнжу нужно делать 2 операции: `operator++` и `operator*`.

`rv::filter` должен:
* при итерировании `operator++` дойти до того элемента, на котором `filter_fn` вернёт `true` (а значит, вызывать `transform_fn` для каждого числа)
* при разыменовании `operator*` нужно вернуть элемент (а значит, снова его вычислить через `transform_fn`). Ни `rv::filter`, ни `rv::transform` не кешируют промежуточных вычислений

Всё становится ещё хуже, если мы захотим конструировать вектор через range:

```c++
// ex. 1
results = v | rv::transform(transform_fn)
            | rv::filter(filter_fn)
            | rs::to<std::vector>();

// ex. 2
rs::push_back(results,
              v | rv::transform(transform_fn)
                | rv::filter(filter_fn));
```

В обоих случаях вывод будем следующим:

```sh
transform is called for 1
transform is called for 2
transform is called for 3
transform is called for 4
transform is called for 5
transform is called for 2  # !!!
transform is called for 3  # !!!
transform is called for 4  # !!!
transform is called for 4  # !!! !!!
transform is called for 5  # !!!
```

**Вопрос**: почему так?

<details>
<summary>ответ</summary>
<p>
    
`to<std::vector>` и `std::ranges::push_back` - оптимизированы для вектора, они вытаются вызвать `reserve` перед вставкой. Для вызова `reserve` они сначала узнают будущий размер вектора (первый проход по всем элементам), затем выполняют обычное итерирование с разыменованием как в первом примере, по-видимому, начиная уже с первого подходящего элемента.
    
</p>
</details>

Автор статей предлагает свои способы исправления проблемы, но в ranges-v3 предусмотрен фикс через особый `rv::cache1` - кеширование результата предыдущего рэнжа в пайплайне:

```c++
results = v | rv::transform(transform_fn)
            | rv::cache1
            | rv::filter(filter_fn)
            | rs::to<std::vector>();
```

соответствующий вывод:

```sh
transform is called for 1
transform is called for 2
transform is called for 3
transform is called for 4
transform is called for 5
```

<br />

**Замечание**: TPOIASI делает использование ranges дороже чем можно предположить: не только оверхед на лишние вызовы и лишние хранения итераторов-функций, но множественное лишнее перевычисление или, если программист озаботился, накладные расходы на кеширование.

**Замечание**: Скорее всего, ranges-специфичный код компилятору будет сложнее оптимизировать чем обычные алгоритмы. На дворе 2020-й, а топовые компиляторы плохо оптимизируют `std::find_if` по `std::vector<int>`-то, не говоря о ranges (см. лекцию о производительности 1 семестра). Это тоже часть платы производительностью за ренжи.

<br />

##### ещё пару полезных примеров

`join`

```c++
std::vector<std::string> words = {"London", "is", "the", "capital", "of", "Great", "Britain"};
std::string sentence = words | rv::join(' ') | rs::to<std::string>;
std::cout << sentence << '.' << std::endl;
```

вывод:

```sh
London is the capital of Great Britain.
```

<br />

`concat` - ленивая (если view) конкатенация входных ренжей

```c++
std::vector<int> v1 = {1, 2, 3};
std::vector<int> v2 = {4, 5, 6};
std::vector<int> v3 = {7, 8, 9};
for (int x : rv::concat(v1, v2, v3))
    std::cout << x;
```

вывод:

```sh
123456789
```

**Вопрос на понимание:** как можно реализовать `concat`?

<br />

`enumerate` - python-like enumerate

```c++
std::vector<std::string> names = { "Dobrynia", "Ilya", "Alesha" };
for (const auto& [ix, name] : names | rv::enumerate)
    std::cout << ix << ": "  << name << std::endl;
```

вывод:

```sh
0: Dobrynia
1: Ilya
2: Alesha
```

**Вопрос на понимание**: как можно реализовать `enumerate`?

<br />

`push_back` - для заполнения `std::vector` с предварительным вызовом `reserve`.

```c++
std::vector<int> v;
rs::push_back(v, rv::iota(1, 10));
```

<br />

##### boost::ranges

[boost ranges (v2)](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/index.html)

Если у вас по какой-то причине ranges-v3 недоступны, можно использовать реализацию ranges из boost. Они же ranges-v2.

Помимо другого namespace, других имён, отстутствия projections и прочего, в boost::ranges нет `action` (в терминологии v3) через `operator|()`. Это связано с одним принципом дизайна boost::ranges:

```
boost range adaptors are always lazy:
...

4. operator|() is used to add new behaviour lazily and never modifies its left argument.
```

В некотором смысле это приятное упущение, т.к. программист явно видит, что при наличии `operator|` все операции ленивые, в то время как в v3 надо начинать думать.

Далее рассмотрим несколько примеров как можно обращаться с boost::ranges. Совсем тривиальные с filter / transform / reverse опустим.

<br />

[`indexed` adaptor (like enumerate in python)](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/adaptors/reference/indexed.html)

```c++
using namespace boost::adaptors;

std::vector<int> input = { /*...*/ };

for (const auto& element : input | indexed(0))
{
    std::cout << "Element = " << element.value()
              << " Index = " << element.index()
              << std::endl;
}
```

<br />

[`indirected` adaptor](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/adaptors/reference/indirected.html)

```c++
using namespace boost::adaptors;

std::vector<std::shared_ptr<int> > input;
for (int i = 0; i < 10; ++i)
    input.push_back(std::make_shared<int>(i));

boost::copy(
    input | indirected,
    std::ostream_iterator<int>(std::cout, ","));
```

<br />

[`map keys`](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/adaptors/reference/map_keys.html) / [`map_values`](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/adaptors/reference/map_values.html)

```c++
using namespace boost::adaptors;

std::map<int,int> input;
for (int i = 0; i < 10; ++i)
    input.insert(std::make_pair(i, i * 10));

boost::copy(
    input | map_keys,
    std::ostream_iterator<int>(std::cout, ","));

boost::copy(
    input | map_values,
    std::ostream_iterator<int>(std::cout, ","));
```

<br />

[`tokenized`](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/adaptors/reference/tokenized.html)

```c++
using match_type = boost::sub_match<std::string::iterator>;

std::string input = " a b c d e f g hijklmnopqrstuvwxyz";
boost::copy(
    input | tokenized(boost::regex("\\w+")),
    std::ostream_iterator<match_type>(std::cout, "\n"));
```

<br />

[`irange`](https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/ranges/irange.html)

```c++
    for (int i : boost::irange(0, 100))  // [0, 100)
        std::cout << i << ' ';
```

<br />

##### boost range algorithms

https://www.boost.org/doc/libs/1_72_0/libs/range/doc/html/range/reference/algorithms.html

Аналогично ranges-v3 в boost range есть алгоритмы, работающие с ренжами.

```c++
std::vector<int> v = { /*...*/ };

boost::sort(v);

boost::remove_erase_if(v, is_even);
```

Полный набор алгоритмов можно найти по ссылке на документацию.

_По возможности пробегитесь по ним глазами на досуге, скорее всего, вы найдёте для себя что-нибудь новенькое, интересное._

<br />

**Резюме**:
* Ranges есть в стандартной библиотеке (since C++20) и в boost (используйте их, если С++20 недоступен), можно поискать ещё реализаций.
* Ranges _при правильном использовании_ делают код более коротким и понятным, чем std-алгоритмы.
* Цена ranges - время компиляции и производительность (мелкие накладные расходы + подводные камни в виде TPOIASI)